In [2]:
import requests
def download_image_from_url(url, filename):
    img_data = requests.get(url).content
    with open(filename, 'wb') as handler:
        handler.write(img_data)

## Define Hyperparameters of the experiment

In [ ]:
## Define Hyperparameters of the experiment
gradio_link = "https://.....gradio.live"

# about the LVLM using AWS Bedrock
aws_access_key_id= "yours_aws_access_key_id",
aws_secret_access_key= "yours_aws_secret_access_key", 
region_name= "yours_region_name"
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0" # model_id of the LVLM

## CECE

In [3]:
import cece
from cece.queries import *
from cece.refine import *
from cece.wordnet import *

import pickle

In [4]:
# import sys
# from BDD100k_classifier import BDD100k_classifier

# classifier = BDD100k_classifier()

from claude_predictor import *

classification_prompt = f"""
Classify each image in their appropriate class according to the driving situation they depict. 
Valid class labels are 'start' or 'stop' and only these, depending on whether the car has to move or stop based on its surroundings.
You need to classify the images in one of these classes.
Pay attention to the semantics that define each class.
Return me only the label of the scene depicted and nothing else.
"""

prompt_analyze = """
Please analyze the images in detail and answer the following question with reason based on these images. 
"""

text_prompt = f"""
Based on your analysis above, classify each image in their appropriate class according to the driving situation they depict. 
Valid class labels are 'start' or 'stop' and only these, depending on whether the car has to move or stop based on its surroundings.
Pay attention to the semantics that define each class.
You need to classify the images in one of these classes.
Return me only the label of the scene depicted and nothing else.
"""


image_names = os.listdir("bdd100k/images/10k/train/")


class Claude_classifier: 
    
    def __init__(self):
        pass
    def classify(self, image_name):
        source_classes = defaultdict(list)
        pred = predict_classes_claude([image_name], source_classes, 
                                                classification_prompt, prompt_analyze, text_prompt, analyze=False)
        pred = pred[image_name][0]
        if pred == "stop":
            return 0
        else:
            return 1

classifier = Claude_classifier()    
classifier.classify("bdd100k/images/10k/train/" + image_names[0])

100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


0

In [6]:
import json 

with open ("BDD100K/bdd100k/labels/sem_seg/rles/sem_seg_train.json") as handle:
    segs = json.load(handle)
    
    
dataset = []
labels = []
index_to_image_id = {}
image_id_to_index = {}

for i, row in tqdm(enumerate(segs["frames"])):
    objs = []
    for obj in row["labels"]:
        objs.append(obj["category"])
    
    dataset.append(objs.copy())
    image_id = row["name"]
    labels.append(classifier.classify(os.path.join("bdd100k/images/10k/train", image_id)))
    index_to_image_id[i] = image_id
    image_id_to_index[image_id] = i
        

7000it [00:00, 41795.74it/s]


In [7]:
def export_text_edits(edits):
    """
    Processes a dictionary of text edits by filtering and transforming elements based on specific criteria.

    The function expects a dictionary with three keys: "transf", "additions", and "removals".
    Each key should map to a list of elements (strings or list of strings).

    The "transf" key is expected to contain a list of tuples, where each tuple represents a pair of word lists.
    The function transforms each tuple by selecting the first word from each list in the pair that does not contain a period.

    For the "additions" and "removals" keys, which map to lists of lists of strings, the function flattens these lists and includes only those elements that do not contain a period.

    Parameters:
        edits (dict): A dictionary containing three keys:
            - "transf": A list of tuples, each containing two lists of words (word pairs).
            - "additions": A list of lists, where each inner list contains words to be added.
            - "removals": A list of lists, where each inner list contains words to be removed.

    Returns:
        dict: A dictionary with the same structure as the input but filtered and transformed based on the criteria:
            - "additions": List of words to be added, filtered to exclude words containing a period.
            - "removals": List of words to be removed, filtered to exclude words containing a period.
            - "transf": A list of transformed word pairs, each selected based on the absence of a period.
    """
        
    transf = []
    for e1, e2 in edits["transf"]:
        ee1, ee2 = None, None
        for e in e1:
            if "." not in e:
                ee1 = e
                break
                
        for e in e2:
            if "." not in e:
                ee2 = e
                break
        transf.append([ee1, ee2])
        
    return {
        "additions": [ee for e in edits["additions"] for ee in e if "." not in ee],
        "removals": [ee for e in edits["removals"] for ee in e if "." not in ee],
        "transf": transf
    }

In [8]:
from cece.xDataset import *
from cece.xDataset import createMSQ

msq_dataset = []
for row in dataset:
    msq = []
    for obj in row:
        try:
            msq.append(connect_term_to_wordnet(obj).union([obj]))
        except:
            try:
                msq.append(connect_term_to_wordnet(obj.replace(" ", "")).union([obj.replace(" ", "")]))
            except:
                pass
    msq_dataset.append(msq.copy())

ds = xDataset(dataset = msq_dataset,
              labels = labels,
              connect_to_wordnet = False)


def get_local_edits(image_id):
    """
    Retrieves and processes local edits between two images in a dataset based on their semantic differences.

    This function identifies the source image by its image_id, then finds the corresponding target image using
    a domain-specific explanation system that prioritizes minimal semantic change (cost). The function retrieves 
    the edits between the source and target images, transforms these edits using the export_text_edits function,
    and compiles the lists of added and removed objects based on these transformations.

    Parameters:
        image_id (int or str): The identifier for the source image in the dataset. This ID is used to find
                               the corresponding source index, and subsequently, the target image and edits.

    Returns:
        tuple: A tuple containing three elements:
            - source_image_id (str or int): The ID of the source image.
            - added_objs (list): A list of objects added in the transition from the source to the target image.
            - removed_objs (list): A list of objects removed in the transition from the source to the target image.

    The function relies on several global data structures:
        - image_id_to_index (dict): Maps image IDs to their respective indices in the dataset.
        - index_to_image_id (dict): Maps dataset indices back to image IDs.
        - ds (object): A dataset object that contains methods to explain differences between images and find edits.
        - labels (list): A list of labels for the images in the dataset, used in the explanation process.
    """
    source_index = image_id_to_index[image_id]
    source_image_id = index_to_image_id[source_index]
    objects_source = [dd for d in ds.dataset[source_index].concepts for dd in d if "." not in dd]

    target_index, cost = ds.explain(ds.dataset[source_index], labels[source_index])
    target_image_id = index_to_image_id[target_index]
    cost, edits = ds.find_edits(ds.dataset[source_index], ds.dataset[target_index])
    
    edits = export_text_edits(edits)
    added_objs = edits["additions"] + [e for [_, e] in edits["transf"]]
    removed_objs = edits["removals"] + [e for [e, _] in edits["transf"]]
    return objects_source, added_objs, removed_objs

## Editor

In [9]:
from edits import Edits
import ast
from editor import Editor
import boto3
from chat import Chat

from PIL import Image
import matplotlib.pyplot as plt

In [10]:
editor = Editor(gradio_link)

## LVLM

In [11]:
# Define the aws runtime and model
bedrock_runtime_client = boto3.client(
    'bedrock-runtime',
    aws_access_key_id=  aws_access_key_id,
    aws_secret_access_key= aws_secret_access_key,
    region_name=region_name
)

model_id = model_id

### Claude decide the edits!

In [18]:
from prompts import prompt_single_step_bdd100k

import os
import shutil

def create_or_replace_dir(directory_name):
    # Check if the directory already exists
    if os.path.exists(directory_name):
        # If it exists, remove it
        shutil.rmtree(directory_name)
    
    # Create the new directory
    os.makedirs(directory_name)

def edit_claude_single_step(image_id):
    
    create_or_replace_dir(f"imgs/bdd100k/claude-haiku/claude/{image_id}")
    source_image_path = f"imgs/bdd100k/claude-haiku/claude/{image_id}/source.jpg"
    
    url = data[image_id]["url"]

    shutil.copyfile(url, source_image_path)
    steps = []
    
    # read the objects 
    objs, added_objs, removed_objs = get_local_edits(image_id)
    
    chat = Chat(model_id, bedrock_runtime_client)

    logs = ""
    excs, i = 0, 1
    orig_label = classify(source_image_path)
    logs += f"Classification: {orig_label}\n"
    new_label = orig_label
    while (new_label == orig_label):
        try:
            prompt = prompt_single_step_bdd100k(objs, added_objs,removed_objs)

            chat.add_user_message_image(prompt, source_image_path) # add a user message with an image and a text prompt
            step = chat.generate().split("-")[0].strip()
            logs += f"\n----\nOutput LVLM: {i}\n{step}\n"
            step = ast.literal_eval(step.split("\n")[0])
            logs += f"Step: {i}\n{step}\n"
            if step[0].lower() == "add":
                new_image, mask = editor.replacer(source_image_path, step[2], step[1])
                if step[1] in added_objs:
                    added_objs.remove(step[1])
                objs.append(step[1])
                steps.append(step)

            elif step[0].lower() == "remove":
                new_image, mask = editor.replacer(source_image_path, step[1], step[2])
                if step[1] in removed_objs:
                    removed_objs.remove(step[1])
                if step[1] in objs:
                    objs.remove(step[1])
                    
                steps.append(step)

            elif step[0].lower() == "replace":
                new_image, mask = editor.replacer(source_image_path, step[1], step[2])
                if step[2] in added_objs:
                    added_objs.remove(step[2])
                objs.append(step[2])

                if step[1] in removed_objs:
                    removed_objs.remove(step[1])
                if step[1] in objs:
                    objs.remove(step[1])
                    
                steps.append(step)
            else:
                print ("Unknown action!")
                return 

            source_image_path = f"imgs/bdd100k/claude-haiku/claude/{image_id}/step_{i}.jpg"
            new_image.save(source_image_path)
            i += 1
            new_label = classifier.classify(source_image_path)
            logs += f"Classification: {new_label}\n"

        except Exception as e:
            excs += 1
            logs += f"Exception: {e}\n"
            if excs >= 5:
                break

    logs += f"\n\n----\n\n{steps}\n\n----\n\n"
    with open(f"imgs/bdd100k/claude-haiku/claude/{image_id}/logs.txt", "w") as handle:
        handle.write(logs)
    return steps
        
    
def classified_as(image_path, cl):
    preds = classifier.classify(image_path)
    if preds == cl:
        return True
    return False 

In [ ]:
from tqdm import tqdm
    
for key in tqdm(list(data.keys())):
    edit_claude_single_step(key)